<a href="https://colab.research.google.com/github/pjmartel/python-for-scientists/blob/master/Introduction_to_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Introduction 

We have learned to make various operations on 2D tabular data in the form of numpy arrays. While those operations can be very powerful tools for extracting all sorts if interesting patterns from the data, there is some some level of awkwardness in the data object itself. For one thing, numpy arrays  are mostly meant to work
with uniform data types, whereas tabular datasets often have heterogeneous types of data. Numpy allow for the creation of *structured arrays* to deal with these heterogeneous data types, but it's just not as simple and immediate as one would hope. Tabular data often contains str fields, categorical values, floats and integers all combined in one very large table with many rows and columns. Rows are very often labelled with headings, very much like tables in Excel worksheets. When analyzing data with these type of tools, it's very often convenient to add new columns to our tables, our delete unneeded ones. Again, this can be done with Numpy arrays, but it's not practical. We could go on with a number of other aspects of data analysis were numpy arrays come short. 

With the very widespread usage of the Python language among data scientists and analysts, it's all but too natural that this issue was addressed in the form of a multi-tooled Python library. This library is called "Pandas" and its constructed around a type of object called "dataframe" that addresses many of the issues raised
by working with tabular data (and also time series data). **Note:** Those familiar with the R programming language will recognize the dataframe type, which resembles its R counterpart in many ways.

#### Basics of working with Pandas

Let's import the pandas library ("pd" is the recommend short-hand alias)

In [0]:
import pandas as pd

Pandas dataframes can be created with the "DataFrame" method in a number of ways.

For instance, they can be created from a dict object, where the keys are column labels and the corresponding  values are lists of data to be contained in each column:

In [0]:
# df = pd.DataFrame({'Name': ['lion','tiger','wolf','beer','panda'],'Avg.weight':})

In [0]:
df = pd.DataFrame({ 'A': [2, 3, 4, 5, 6] , 'B': [4, 9, 16, 25, 36]})

Let's evaluate the dataframe "df" in a cell

In [0]:
df

As previously explained , the keys in the dictionary have been used for *column labels*, and the lists of values  as column data. Notice a leftmost column starting at 0 and going up to 4  - this is the index column of the data frame and can be used to access and manipulate the data rows in different ways - when not specifically set, his column will consist of numbers ranging from zero to the `n-1`, where `n` is the number of rows.. It is important to understand that the index column and the labels row are not part of the data proper, they are just entry points to label and access the data in ways that are simpler and faster than what you could do with a 2D Numpy array.  Also, the python notebook knows how to "pretty print" the dataframe in a nice tabular way,  with data properly aligned - it's far more readable that the default numpy array representation. 
<br><br>
Suppose we want to print out the values of column A. We can do this:

In [0]:
df['A']

The column values get printed along with indices, and the column data type is indicated below (Remember: dataframes will often include columns of different data types).

Column referencs like the one above can be operated by methods similar to those numpy arrays:

In [0]:
df['B'].mean()

The  column labels work much in the same way as row indices in a numpy 2D array, with the added convenience of (possibly) illustrative names. However,  this comes at a price: you can no longer access elements directly with slice notation. Just try it and see what happens:

In [0]:
df[3,2]

We got a "KeyError", because direct indexing of dataframes is done by column labels.  If you need to refer to elements of the dataframe using numpy-like slice notation, you should  use the "iloc" method:

In [0]:
df.iloc[1:3,1]

It is very easy to add new columns of labelled data to a dataframe:

In [0]:
df['C'] = [8,9,3,5,6]
df

Simple as that! This is now clearly beyond normal numpy array territory  (although it's possible to do something similar with numpy arrays, it's much more convoluted) 

As previously mentioned, dataframes are prepared to handle a mix of different data types. Let's add another column, this time with str data instead of numbers:

In [0]:
df['D']= ['aaa','b','ccc','dd','eeeee']
df

The type structure of a dataframe may easily be checked by inspecting its "dtypes" attribute:

In [0]:
df.dtypes

Operations on dataframe rowns and columns can be done similarly to numpy, using methos like "sum" or "average" and the "axis" parameter to select rowise or columnwise operations. For instance;

In [0]:
df.sum(axis=1)

or df.sum()

Notice that pandas simply ignores the str column when calculating the averages along rows. For rowise computation one can simply do:

In [0]:
df.sum()   # same as df.sum(axis=0)

(notice string sum rule being applied to column D)

Let's create a new column, called "SumBC", containing the sum of values in columns B and C:

In [0]:
df['SumBC'] = df.iloc[:,1:3].sum(axis=1)

(Remenber: slice notation "`[:,1:3]`" means "any row, columns 1 to 2")

In [0]:
df

Now lets assume we want to create a another column, "SumAC", containing the sum of columns "A" and "C". We can do this in a very readable manner using the "loc" indexer object:

In [0]:
df['SumAC'] = df.loc[:,['A','C']].sum(axis=1)   # Why do we need to use the ":" symbol ?
df

While "`iloc`" selects values in the dataframe based on row and column position (pretty like slice notation in numpy arrays), the  "`loc`" object indexer  creates selection based on index and column *labels*. This selections can als support slice syntax, for instance:


In [0]:
df.loc[1:3,'B':'D']

**HANDS-ON:** Can you spot one important difference between the way slicing works for position based and label based ?

In [0]:
#@title ANSWER
# a position range slice [a:b] means from a to b-1, while a label range
# slice [a:b] means from label(or index) a to label (or index) b

In [0]:
df

Removing data from the dataframa is as easy as adding it. Let's say we don't need the "D" column anymore:


In [0]:
df.drop(columns=['D'])

We can also list and compute values in a conditional manner, combining logical expressions with indexing. Can yoy guess what the below expression does ?

In [0]:
df['CondSum'] = df.loc[df['C']>6,'A':'C'].sum(axis=1)

In [0]:
df

That's right, this sums columns A to C and places the results in column "CondSum", but ONLY if the value in column C is above 6. When that condition is not met, there's nothing to place in the corresponding position of the "CondSum" column, and a "NaN" (Not a Number) value is stored there. We can think of NaN as a kind
of placeholder that pandas uses when there no data available to fill that position. NaN will pop up when reading partially filled tabular data from a file, for instance. The missing values will appear as NaN in the dataframe. 

You can use "`loc`" in a sligntly different way to create columns based on conditions:

In [0]:
df.loc[df['A'] % 2 == 0, 'IsEven'] = True
df

Notice the NaN where nothing was stored because the condtion wasn't met. We could fix it with a second line:

In [0]:
df.loc[df['A'] % 2 != 0, 'IsEven'] = False

**HANDS-ON**: Actually, this a somewhat contrived example, we could have also done the same thing with just one line. Can you guess how ?

In [0]:
#@title SOLUTION
df['IsEven2'] = df['A'] % 2 == 0
df

#### Importing and working with a data file


Ok, so we went through some basic operations on tabular data stored as a panda dataframe. In real world scenarios, these tabular data often need to be read from an external site, like a file on a web server or a remote disk. Rather than manually creating a dataframe from a `dict` object like we just did, we will want to 
read dataframes from large files that may have thousands of lines and columns. Fortunately for us, pandas data frames can be created from files in many different formats, including (but not limited to):



*   CSV (comma-separated value)
*   Excel
*  Json
* HTML
* SQL

<br>
To illustrate this functionality, we are going to read a file in .csv format into a dataframe. As the name implies, csv files contain multiple lines with data items separated by commas. They often (but not necessarily) include a header line to be used as column labels, and a leftmost row label column. Obviously, it is up
to the user to decide how such lines should be read - some knowledge on the organization of the imported dataset is usually necessary for proper data preparation and analysis. 

The csv data we are going to read is at the following link: https://raw.githubusercontent.com/pjmartel/python-for-scientists/master/datasets/grades_dataset.csv

You can browse it by mousing over the link, clicking the right button and selecting the option "open in new window". You will recognize these data as a slightly more embellished version of the course grades we used in our numpy introduction. (Student names sound familiar ?... ).

Let's read these data into a pandas dataframe using the appropriately named `read_csv` function:

In [0]:
grades = pd.read_csv("https://raw.githubusercontent.com/pjmartel/python-for-scientists/master/datasets/grades_dataset.csv")

In [0]:
grades

**HANDS-ON:** You have a number of tasks to perform with this dataset:



1.   You realize that the table is missing the grades of two of the students! The names are "Tyrion" and "Bronn" and their grades are respectively `[15.2,  16.7, 16.0]` and `[17.0,15.2,17.7]`. Please add the missing grades! (Hint: consider using the "`loc`" indexer)
2.   Calculate the student grade average of the three tests and place the values on a column labeled "Average"
3.   Calculate the grade average for each test and place the value on a bottom row labeled "test_averages"
4.  Create a new column named "Status" containing "Pass" or "Fail" depending on the grade average being higher or lower than 9.5. (Hint: look at the conditional expression above)
5. Some of the students who have an above 9.5 grade average will nevertheless fail because they a had a grade < 7 in at least one of the three tests. Write an 
expression to correct the "Status" column accordingly (Hint: conditionals are again your friends, what does "`a or b or c`" evaluate  to when at least one of a, b and c is `False` ?)
6. Sort the data based on the column "Name" (Hint: search for sort methods for a data frame, using TAB completion or the "`dir`" command)


In [0]:
#@title SOLUTION 1.
grades.loc[11] = ['Tyrion',15.2,16.7,16.0]
grades.loc[12] = ['Bronn',17.0,15.2,17.7]
grades
# Note: you must an index value outside the range 0-11 or else will 
# overwrite data rows (tow rows cannot have the same index)

In [0]:
#@title SOLUTION 2.
grades['Average'] = grades.iloc[:,1:4].mean(axis=1)
grades

In [0]:
#@title SOLUTION 2. (alternative solution)
grades['Average'] = grades.loc[:,'1st Test':'3rd Test'].mean(axis=1)
grades

In [0]:
#@title SOLUTION 3.
grades.loc['test_averages'] = grades.iloc[:,1:5].mean(axis=0)
grades

In [0]:
#@title SOLUTION 4. 
grades.loc[grades.Average >= 9.5, 'Status'] = 'Pass'
grades.loc[grades.Average < 9.5, 'Status'] = 'Fail'
grades

In [0]:
#@title SOLUTION 4. (alternate solution)
import numpy as np
grades['Status'] = np.where(grades['Average']>=9.5, 'Pass', 'Fail')
grades
# Can you understand how this works ?

In [0]:
#@title SOLUTION 5.
grades.loc[ (grades['1st Test']<7.0) | (grades['2nd Test']<7.0) | (grades['3rd Test']<7.0),'Status'] = 'Fail'
grades

In [0]:
#@title SOLUTION 6.
grades = grades.sort_values('Name') 
# by default, sort_values is not an in place operation!
grades

In [0]:
#@title SOLUTION 6. (alternative solution)
grades.sort_values('Name',inplace = True)
grades

Ok, if you solved the Hands-On properly, you should be left with the data frame in this state:

In [0]:
grades

Dataframes have inbuilt plotting functions (based on matplotlib), which allow for fast and convenient ways of visualizing the dataframe data.  The plot method:

#### Dataframe plotting methods



In [0]:
grades.plot()

will produce a list of line plots, one for each column. On the x axis we have index values, and on the y axis column data. With this plot it's very easy to see that students had a very consistent performance across the three tests  (students 1 and 8 appear to have had special trouble with the 3rd test).

Dataframes also have a convenient "hist" method for producing histograms:

In [0]:
grades.hist();

The plot method has several sub-methods for specific plot styles. Let's suppose we wanto to plot 1st test grades against averages:

In [0]:
grades.plot.scatter(x='1st Test',y='Average')

The "plot" method accepts a variety of parameters that can be passed onto the underlying matplotlib routines, like "kind", style" and "mfc" (marker face color):

In [0]:
grades.plot(kind="line", style='go-',mfc='black')

#### Exporting your dataframe to the Google Drive



Now that we haver prepared our "grades" dataframe with all the computations and columns, we may want to save it to a file, in a number of formats.

Here we will see how to save to  a folder  in your google drive, in the Excel format.

First, we need to *mount* our Google Drive on the VM file system. As explained in the "reading and writing files" notebook, we will use the google colab api with
the authentication token mecanism:

In [0]:
from google.colab import drive

In [0]:
drive.mount('/gdrive')


Now that our Google drive is mounted at mountpoint "/gdrive", let's export  our data frame to an Excel "xlsx" file using the dataframe method "`to_excel`" :

In [0]:
grades.to_excel("/gdrive/My Drive/grades_colab.xlsx")

Let's check that the file is there:

In [0]:
%ls /gdrive/My\ Drive

Now go to your Google Drive and try to open the "grades_colab.xlsx" file with Google Spreadsheets. 